In [1]:
import pandas as pd
import os
import time
cwd = os.getcwd()


In [2]:
#set NRCELL output file from parser
nrcell_file = 'C:/Work/NPO PM/Optus Project/04 Brisbane Leadership/01 SmartCAPEX/00 Optus Info/Parameter and Counter//RC16_NRCEL_20221018_orig.xlsx'

#Dataframe for NRCELL sheet
df1 = pd.read_excel(nrcell_file,sheet_name='NRCELL',skiprows=1)
col_list = df1.columns.str.strip()
df1.columns = col_list
selected_col_list = ['MRBTS','NRBTS','NRCELL','chBw', 'nrarfcn','dlMimoMode','frRefList','rModDNList','Item-tddFrameStructure-ulDlDataSlotRatio','cellName','administrativeState',
 'operationalState']
NRCELL_df = df1[[c for c in df1.columns if c in selected_col_list]]



In [3]:
#Dataframe for NRCELL_FDD sheet
df2 = pd.read_excel(nrcell_file,sheet_name='NRCELL_FDD',skiprows=1)
col_list = df2.columns.str.strip()
df2.columns = col_list
df2.iloc[ : , [0,1,2,6,8] ]
NRCELL_FDD_df = df2
NRCELL_FDD_df['DN_MRBTS'] = 'PLMN-PLMN/MRBTS-' + NRCELL_FDD_df['MRBTS'].astype(str) + '/NRBTS-' + NRCELL_FDD_df['NRBTS'].astype(str)
NRCELL_FDD_df['DN_NRCELL'] = NRCELL_FDD_df['DN_MRBTS'] + '/NRCELL-' + NRCELL_FDD_df['NRCELL'].astype(str)

In [4]:
#Dataframe for RMOD_R sheet
df3 = pd.read_excel(nrcell_file,sheet_name='RMOD_R',skiprows=1)
col_list = df3.columns.str.strip()
df3.columns = col_list
selected_c1_list = ['configDN','productCode','productName']
RMOD_df = df3[[c for c in df3.columns if c in selected_c1_list]]
RMOD_df[['productShortCode','tmp']] = RMOD_df['productCode'].str.split('.',expand=True)
RMOD_df = RMOD_df.drop(columns=['tmp'])

C:\Users\wongthot\Anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [5]:
df4 = pd.read_excel(nrcell_file,sheet_name='ASIRMOD_R',skiprows=1)
col_list = df4.columns.str.strip()
df4.columns = col_list
selected_c2_list = ['configDN','productCode','productName']
ASIRMOD_df = df4[[c for c in df4.columns if c in selected_c2_list]]

In [6]:
#NRCELL_df[NRCELL_df['rModDNList'].isna()].count()

In [7]:
[col for col in df1.columns if 'state' in col.lower()]

['administrativeState',
 'energySavingState',
 'operationalState',
 'blockingState']

In [9]:
#Dataframe for product derived from RMOD_R
product_df = RMOD_df[['productCode','productName']].drop_duplicates().copy()
product_df[['productShortCode','tmp']] = product_df['productCode'].str.split('.',expand=True)
product_df = product_df.drop(columns=['tmp','productCode'])[::-1]
product_df = product_df.drop_duplicates().dropna().set_index('productShortCode')


In [10]:
RMOD_df = pd.merge(RMOD_df,product_df,on='productShortCode',how='left')
RMOD_df['productName_x'] = RMOD_df['productName_y'].fillna(RMOD_df['productName_x'])
RMOD_df = RMOD_df.drop(columns=['productShortCode','productName_y'])
RMOD_df.columns = ['configDN','productCode','productName']

In [11]:
#if 'dlMimomode' in df.columns:
#    print('yes')
#else:
#    print('no')

In [12]:
#[col for col in df.columns if 'tdd' in col]

In [13]:
NRCELL_df['DN_MRBTS'] = 'PLMN-PLMN/MRBTS-' + NRCELL_df['MRBTS'].astype(str) + '/NRBTS-' + NRCELL_df['NRBTS'].astype(str)
NRCELL_df['DN_NRCELL'] = NRCELL_df['DN_MRBTS'] + '/NRCELL-' + NRCELL_df['NRCELL'].astype(str)


C:\Users\wongthot\AppData\Local\Temp/ipykernel_8008/1172180520.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NRCELL_df['DN_MRBTS'] = 'PLMN-PLMN/MRBTS-' + NRCELL_df['MRBTS'].astype(str) + '/NRBTS-' + NRCELL_df['NRBTS'].astype(str)
C:\Users\wongthot\AppData\Local\Temp/ipykernel_8008/1172180520.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NRCELL_df['DN_NRCELL'] = NRCELL_df['DN_MRBTS'] + '/NRCELL-' + NRCELL_df['NRCELL'].astype(str)


In [14]:
#Generate SiteName, Tech, SectorName for Smart Capex
NRCELL_df[['cellName','cell']] = NRCELL_df['cellName'].str.split(':',expand=True)
NRCELL_df[['SiteName','Tech','Sec','Sector']] = NRCELL_df['cell'].str.split('-',expand=True)
NRCELL_df['SectorName'] = NRCELL_df['SiteName'] + '_' + NRCELL_df['Sector']
NRCELL_df = NRCELL_df.drop(columns=['cellName','cell','Sec','Sector'])

C:\Users\wongthot\AppData\Local\Temp/ipykernel_8008/4048788541.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NRCELL_df['SectorName'] = NRCELL_df['SiteName'] + '_' + NRCELL_df['Sector']


In [15]:
#Combine frRefList and rModDNList
NRCELL_df['DNList'] = NRCELL_df['frRefList'].fillna(NRCELL_df['rModDNList'])
NRCELL_df[['Temp','configDN','RMOD2']] = NRCELL_df['DNList'].str.split(';',expand=True)
NRCELL_df = NRCELL_df.drop(columns=['frRefList','rModDNList','DNList','Temp'])

In [16]:
#NRCELL_df = NRCELL_df.iloc[: , :-1]

In [17]:
#NRCELL_df[NRCELL_df['configDN'].notnull()]

In [18]:
#Merged Dataframe
merged_df = pd.merge(NRCELL_df, NRCELL_FDD_df[['DN_NRCELL','chBwDl','nrarfcnDl']],on='DN_NRCELL', how='left')
merged_df['chBw']= merged_df['chBwDl'].fillna(merged_df['chBw'])
merged_df['nrarfcn']= merged_df['nrarfcnDl'].fillna(merged_df['nrarfcn'])
merged_df = merged_df.drop(columns=['chBwDl','nrarfcnDl'])
merged_df = pd.merge(merged_df,RMOD_df, on='configDN', how='left')
merged_df = pd.merge(merged_df,ASIRMOD_df, on='configDN', how='left')
merged_df['productName_x']= merged_df['productName_y'].fillna(merged_df['productName_x'])
merged_df = merged_df.drop(columns=['productCode_y','productName_y'])
merged_df = merged_df.rename(columns={'productCode_x': 'productCode', 'productName_x': 'productName'})

In [19]:
TodaysDate = time.strftime("%Y%m%d")
excelfilename = 'NRCELL_for_Smart_Capex_' + TodaysDate +".xlsx"
merged_df.to_excel(excelfilename,sheet_name='NRCELL_Smart_Capex')